<font size="1"> *This notebook is best viewed in jupyter lab/notebook. You may also choose to use Google Colab but some parts of the images/colouring will not be rendered properly.</font> 

<div class="alert alert-block alert-warning">

# <p style="text-align: center;">Lab 4 (Weeks 8,9): Convolutional Neural Networks (CNNs)</p>
## <p style="text-align: center;">Notebook II: Applications</p>

<img src="https://miro.medium.com/v2/resize:fit:592/1*V6Y8FF2qfw_ztNbs1AHXNg.png" width="800" height="400" />

<!-- ![linear-vs-logistic-regression--medium](https://miro.medium.com/max/1400/1*dm6ZaX5fuSmuVvM4Ds-vcg.jpeg) -->

Welcome to the fourth and <b>final</b> lab! As usual, this lab will span over two weeks.
            
In this Lab, you will find four tasks distributed across two Jupyter Notebooks: *Lab4_Basics_student.ipynb* and *Lab4_Applications_student.ipynb*. The first two tasks guide you to train a basic CNN using pytorch lightning. In the last task, you will apply the knowledge you gained to solve a practical problem.
    
- <b>Task 1:</b> Design a CNN and train to classify STL-10 images
- <b>Task 2:</b> Analyse the results on the STL-10 test images
- <b>Task 3:</b> Design and train a CNN by yourself on the FER-4 dataset and analyse results
      
Each task will contain code to complete, and a worded question, so ensure you complete both before submitting. Feel free to add your own additional comments.
    
After completion, You need to submit both Jupyter Notebooks (.ipynb files) to Moodle. Make sure all the outputs are visible before submitting.
    
Good luck with the final Lab! Submit it before the <b>deadline</b> to enjoy full marks.

__Submission details:__
- __Make sure you have run all your cells from top to bottom (you can click _Kernel_ and _Restart Kernel and Run All Cells_).__ </br>
- __Submit the Jupyter Notebooks (Lab4_Basics.ipynb_) and (Lab4_Applications.ipynb_).__
- __Outputs must be visible upon submission. We will also be re-running your code__

<b>Enter you student details below</b>

- <b>Student Name:</b> Firstname Lastname
- <b>Student ID:</b> 123456789   

<div class="alert alert-block alert-danger">

## Table of Contents
    
* [Libraries](#Libraries)

* [Task 3: Design and train a CNN by yourself on the FER-4 dataset and Analyse Results](#task_3)

* [Discussion Questions](#t3_1)

<div class="alert alert-block alert-warning">

# Libraries

In this lab, you will use several pytorch and pytorch lightning libraries along with several other basic python libraries. All the libraries that you need are given below.

In [ ]:
import os
import random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.image as mpimg

import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader, random_split
from torch.nn.functional import log_softmax, softmax
import torchvision
import torchvision.transforms.functional as F
from torchvision import transforms
import torchmetrics
from torchsummary import summary

from IPython.display import display

import lightning as pl ## Pytorch lightning is a wrapper for pytorch that makes it easier to train models
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import Callback, ModelCheckpoint, EarlyStopping
from lightning.pytorch.callbacks.progress import TQDMProgressBar, RichProgressBar
from lightning.pytorch.callbacks.progress.rich_progress import RichProgressBarTheme

# Setting seeds for reproducibility
pl.seed_everything(4179)
random.seed(4179)
np.random.seed(4179)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

<div class="alert alert-block alert-warning">

# Facial Expression Recognition (FER) Dataset <a class="anchor" id="python-basics"></a>
    
For all the work in this notebook, you will be using the FER dataset.
    
<img src="https://production-media.paperswithcode.com/datasets/FER2013-0000001434-01251bb8_415HDzL.jpg" width="500" />
    
The FER dataset contains diverse grayscale images of 7 facial emotions classes: anger, disgust, fear, happiness, sadness, surprise, and neutrality. Each image is of size 48x48 pixels, enabling efficient processing. For simplicity, in this task you will be using a subset of the full dataset which only contains the four classes (0)happy, (1)neural, (2)sad, and (3)surprise. We will call this the **FER-4 dataset**.

<div class="alert alert-block alert-info">

# Task 3 - Design and train a CNN on the FER-4 dataset and Analyse Results <a class="anchor" id="task_3"></a>
        
In this task, we do not enforce any model architecture or hyper-parameters (only some recommendations). You will design a CNN for image classification task and train it on the FER-4 dataset. You will use PyTorch's inbuilt datasets class, and Pytorch Lightning's module class to construct a CNN in order to perform training on the FER-4 dataset. You should use the knowledge obtained from Task 1 to complete this task. 

Note that you may want to use *transforms.Grayscale(num_output_channels = 1)* to convert the .jpg images to grayscale tensors. You will also analyse the results on the test dataset using the knowledge obtained from Task 2.
    
**Final deliverables of this task will be the following:**
1. Accuracy of more than 65% on the test dataset.
2. Reasonably converging train/val loss/accuracy curves.
3. Confusion Matrix
4. Visualization of top 5 misclassified images for each of the four classes.
5. Visualization of the saliency map of a correctly predicted 'happy' image.
6. Use an image of yourself! Capture your image from the webcam/phone and test your model.

We have provided headings for where things should go, but feel free to add more to meet all the final deliverables!

### Let's take a quick look at the dataset

In [ ]:
########################################################################
# DO NOT CHANGE

# Get a list of class names from the 'data/fer4/train' directory
class_names = os.listdir('data/fer4/train')

# Print the list of class names
print(f'class names: {class_names}')

# Calculate the number of classes by counting the elements in the 'class_names' list
num_classes = len(class_names)

# Print the number of classes
print(f'number of classes: {num_classes}')
########################################################################

### Define Transformations

In [ ]:

train_transforms = transforms.Compose([
    ???? # Convert the data to tensor
    transforms.Grayscale(num_output_channels=1), # Transform it to a single channel grayscale image
    ???? # Resize the image to 48x 48
    ???? # Normalize pixel values to have a mean and standard deviation of 0.5
])


val_test_transforms = transforms.Compose([
    ???? # Convert the data to tensor
    ???? # Transform it to a single channel grayscale image
    ???? # Resize the image to 48x 48
    ???? # Normalize pixel values to have a mean and standard deviation of 0.5
])
########################################################################

### Create Datasets

For this dataset, we use a different way to create the dataset. Since we already have the images ready in the folder, we will just create them using the ```torchvision.dataset.ImageFolder``` method.

In [ ]:
########################################################################
# DO NOT CHANGE
trainset = torchvision.datasets.ImageFolder(root='data/fer4/train', transform=train_transforms)
trainset, valset = torch.utils.data.random_split(trainset, [0.7, 0.3])
testset = torchvision.datasets.ImageFolder(root='data/fer4/test', transform=val_test_transforms)

print('Image count for each set\n------------------------')
print(f'trainset\t: {len(trainset)}')
print(f'valset  \t: {len(valset)}')
print(f'testset \t: {len(testset)}')
########################################################################

### Create Dataloaders

In [ ]:
BATCH_SIZE = ??? # Define batch size

trainloader = ???
valloader = ???
testloader = ???

print('Batch shape for each loader\n---------------------------')
images, labels = next(iter(trainloader))
print(f'trainloader\t: {images.shape}')
images, labels = next(iter(valloader))
print(f'valloader  \t: {images.shape}')
images, labels = next(iter(testloader))
print(f'testloader \t: {images.shape}')

### Visualize the dataset

Make sure you always get a feel for the dataset before you start applying models to it!

In [ ]:
# Write a function to visualize images from a given data loader 
# Note: Use titles to denote class label in words.
def visualize_dataloader(dataloader, class_names):
    
    ???
    
# visualize images from the trainloader
print('train set')
visualize_dataloader(trainloader, class_names)

# visualize images from the valloader
print('val set')
visualize_dataloader(valloader, class_names)

# visualize images from the valloader
print('test set')
visualize_dataloader(testloader, class_names)

<div class="alert alert-block alert-info">

### Design your own CNN

Here are some guidelines in the CNN you can construct (you may stray away from this if you find other architectures are better):
- 4 convolutional layers
- Some (or all) of the convolutional layers can have pooling layers (this reduces size of feature maps well!)
- Test several activation functions (ReLU, sigmoid, tanh etc.) 
- Test different optimizers (SGD usually uses lr=0.1, ADAM usually uses lr=0.001, these are only guidelines and you can tweak the learning rates based on the datasets)

In [ ]:
class MyCNN(pl.LightningModule):
    
    def __init__(self, num_classes):
        super().__init__()
        
        ???


    def forward(self, x):
        
        ???
        
        return ???

    def training_step(self, batch, batch_idx):
        # Define logic for training step
        
        ???
        
        return ???

    def validation_step(self, batch, batch_idx):
        # Define logic for validation step
        ???

    def test_step(self, batch, batch_idx):
        # Define logic for test step
        ???
        
    def predict_step(self, batch, batch_idx):
        # Define logic for inference/prediction step
        ???

        return ???

    def configure_optimizers(self):
        # Configure optimizers and schedulers
        ???
        return ???

    ####################
    # DATA RELATED HOOKS
    ####################

    def train_dataloader(self):
        return trainloader

    def val_dataloader(self):
        return valloader

    def test_dataloader(self):
        return testloader

### Initialize the CNN

In [ ]:
task3_model = ??? # intialize CNN

# print summary of the model to double check
summary(task3_model.to('cuda'), (1, 48,48)) # delete .to('cuda') if not using cpu

### Define progress bar and checkpoint callback functions 

In [ ]:
# Define checkpoint callback function to save the model at the best epoch
checkpoint_callback = ModelCheckpoint(
    monitor="val_acc",
    save_top_k=1,        # save the best model based on validation accuracy
    mode="max",
    every_n_epochs=1
)

### Train the CNN

In [ ]:
task3_trainer = ??? # Call pl.Trainer and put in the relevant arguments

task3_trainer.fit(task3_model)

### Test the CNN

For a propoer trained network, you may achieve an accuracy for at least 65%

In [ ]:
task3_trainer.test(task3_model) 

### Read logs

In [ ]:
metrics_task_3 = ????
metrics_task_3.set_index("epoch", inplace=True)
metrics_task_3 = metrics_task_3.groupby(level=0).sum().drop("step", axis=1) 

### Plot train and validation losses against epoch

In [ ]:
????

### Plot train and validation accuracies against epoch

In [ ]:
???

### Get predictions for the test set for later use 

In [ ]:
## These are the predictions for the test set. You will use this for the tasks below during the analysis process.

predictions = task3_trainer.predict(task3_model, testloader) # do not do anything to the variable 'predictions' you will reuse it

test_outputs = torch.concat([prediction[0] for prediction in predictions], dim=0)
test_labels = torch.concat([prediction[1] for prediction in predictions], dim=0)
test_inputs = torch.concat([prediction[2] for prediction in predictions], dim=0)
test_preds = ???? # Get the predicted lables

### Confusion Matrix

Ensure you use **proportion** instead of absolute value. Create the confusion matrix for the four classes.

In [ ]:
# Plot the confusion matrix
# You can create a ConfusionMatrix instance for multiclass classification with 'num_classes' from the lightning library

???

### Top Misclassified

You will only plot the top misclassified classes here. Please plot 5 images of each class (so 20 images in total). This will be similar to the basics notebook.

In [ ]:
???

### Saliency Map

Plot a saliency map corresponding to a happy face that was classified correctly

In [ ]:
???

### My own image

Use an image of yourself being happy - hopefully you are proud of your results! Make sure you output the predicted result

In [ ]:
# Read in your image here (we used mpimg.imread and you just need to pass in the path to your image)
# Feel free to use any other image reading functions instead
img = mpimg.imread(???)[:,:,0:3]  # get rid of the 4th axis if exists
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Pass your image through the network
# Remember to apply the val_test_transforms for your image before doing a forward pass of your model

???

<div class="alert alert-block alert-success">

## Discussion Questions <a class="anchor" id="t3_1"></a>
    
#### Comment on the resulting confusion matrix.
    
Answer: 

#### What was the strange thing that you observed in the actual labels when visualizing the misclassified images?
    
Answer: 
    
#### Comment on the saliency map of the 'happy' image.
    
Answer: 
    
#### Comment on the prediction results of your own data.
    
Answer:

<div class="alert alert-block alert-danger">

#### This is the end of Notebook II. Hurrah! You have now completed all the Labs for this unit! It's time to reward yourself with an assignment!!!